In [ ]:
# Load Data
import pandas as pd
import ray

#ray.shutdown()
#ray.init(object_store_memory=4000000000) # set object store memory to 4GB

sample = pd.read_pickle('./sample.pkl')


# tansform 'Rating' from Categorical to float
import numpy as np
sample["Rating"] = pd.to_numeric(sample["Rating"], errors='coerce')

# drop all rows with inf/-inf values!
import numpy as np
sample = sample[(sample != np.inf).all(axis=1)]
sample = sample[(sample != -np.inf).all(axis=1)]

#get rid of whitespace to draw tree later
Eq_Stylebox = sample['Eq_Stylebox_Long'].astype('object').replace(' ','_', regex=True)
sample['Eq_Stylebox_Long'] = Eq_Stylebox.astype('category')

# shifting target variable to predict next month
sample['returns'] = sample.returns.shift(-1)
sample = sample.drop(sample.tail(1).index)


X = sample.drop('returns', axis=1)
y = sample['returns']

# create dummies in case of categorical data
dummy_needed = [#'Rating',
                'Financial_Health_Grade_Long',
                 'Growth_Grade_Long',
                 'Profitability_Grade_Long',
                 'Eq_Stylebox_Long']

X = pd.get_dummies(X, columns=dummy_needed)

# Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 26)
X_train = ray.put(X_train)
y_train = ray.put(y_train)

### Distributed Ray XGB
https://github.com/ray-project/xgboost_ray

In [ ]:
import time
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.hyperopt import HyperOptSearch
from xgboost_ray import RayDMatrix, RayParams, train
from ray import tune

def train_xgboost(config, checkpoint_dir=None):
    start_time = time.time()
    
    train_set = RayDMatrix(X_train, y_train)
    test_set = RayDMatrix(X_test, y_test)
    
    evals_result = {}
    
    # train the model
    bst = train(
        params=config,
        dtrain=train_set,
        evals_result=evals_result,
        evals=[(test_set, "eval")],
        verbose_eval=False,
        num_boost_round=5, #equivalent to 'epochs'
        ray_params=RayParams(num_actors=4, cpus_per_actor=2)) #parameters for parallelism
    
    model_path = 'model.xgb'
    bst.save_model(model_path)
    print(f'total time taken: {time.time()-start_time}')
    print('Final rmse: {:.4f}'.format(
    evals_result["eval"]["rmse"][-1]))
    
    return bst
    

# Define the search algorithm and scheduler
#### NOT SURE IF NEEDED OR SUPPORTED IN XGB_RAY! maybe uncomment here and in config
search_alg = HyperOptSearch()
scheduler = ASHAScheduler(max_t=10, grace_period=1)

# Specify the hyperparameter search space.
config = {
    "tree_method": "approx", # if access to GPU, set to 'gpu_hist'
    "objective": "reg:squarederror", 
    "eval_metric": "rmse", 
    "learning_rate": tune.loguniform(1e-4, 0.9),
    "subsample": tune.uniform(0.8, 1.0),
    "colsample_bytree": tune.uniform(0.8, 1.0),
    "colsample_bylevel": tune.uniform(0.8, 1.0),
    "max_depth": tune.randint(1, 16),
    "n_estimators": tune.randint(500, 10000),
    "reg_alpha": tune.loguniform(1e-3, 100),
    "reg_lambda": tune.loguniform(1e-3, 100),
}

# Run the hyperparameter search
analysis = tune.run(
    train_xgboost,
    resources_per_trial=RayParams(num_actors=4, cpus_per_actor=2).get_tune_resources(),
    config=config,
    num_samples=5,
    search_alg=search_alg,
    scheduler=scheduler,
    metric='rmse',
    mode='min',
    verbose=1
)

best_RMSE = analysis.best_result["rmse"]
print(f'Best model parameters: {analysis.best_config}')
print(f'Best RMSE: {best_RMSE}')
print(analysis.best_config)
